In [1]:
#Load libraries
import numpy as np
from sklearn.decomposition import PCA
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold

## First Analaysis with CNN features and PCA (28.04)

In [42]:
import pickle
with open(r"training_set.p", "rb") as input_file:
    data_full = pickle.load(input_file)

In [43]:
# Input data here
y_full = data_full[1]
x_full = data_full[0]
#Label and x-data
#y_full =  np.random.binomial(1, 0.5, size=600)
#x_full = np.random.rand(600,750)
pd.DataFrame(y_full).describe()
#pd.DataFrame(x_full).describe()

,0
count,863.000000
mean,0.492468
std,0.500233
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [44]:
#Try PCA first
pca_all = decomposition.PCA(n_components = 100) #use 150 if you want more variance explained
pca_all.fit(x_full)


PCA(n_components=100)

In [45]:
#get new X
x_full = pca_all.transform(x_full)

In [46]:
pd.DataFrame(x_full).describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,863.000000,863.000000,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,8.630000e+02,...,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02
mean,0.000004,-0.000005,-6.257451e-08,-0.000003,-3.847020e-07,9.402754e-07,-0.000001,4.510752e-07,-4.374691e-07,7.488740e-07,...,4.994220e-07,-4.147116e-07,6.659852e-07,6.203450e-07,2.612624e-07,1.067773e-07,1.802643e-07,4.038508e-07,4.660714e-08,5.174743e-07
std,2.903462,1.271854,1.227888e+00,1.146522,9.209777e-01,8.445960e-01,0.762597,6.650895e-01,6.480259e-01,5.509892e-01,...,8.131318e-02,7.999986e-02,7.933204e-02,7.841248e-02,7.791001e-02,7.743447e-02,7.550553e-02,7.461286e-02,7.363985e-02,7.229830e-02
min,-5.924027,-3.423488,-4.489534e+00,-2.888274,-3.092031e+00,-3.133672e+00,-2.651993,-2.150130e+00,-1.800071e+00,-1.475130e+00,...,-3.557998e-01,-3.648895e-01,-3.113829e-01,-2.638743e-01,-3.214611e-01,-2.836448e-01,-2.599582e-01,-2.312940e-01,-2.469697e-01,-2.417980e-01
25%,-2.247991,-0.767917,-8.127629e-01,-0.778934,-6.250603e-01,-5.000992e-01,-0.469976,-4.550316e-01,-4.141276e-01,-3.813925e-01,...,-4.787713e-02,-5.291305e-02,-4.490145e-02,-5.110228e-02,-4.812619e-02,-4.680119e-02,-4.797791e-02,-4.523966e-02,-4.557248e-02,-4.175265e-02
50%,-0.179334,-0.083924,8.521469e-02,-0.130936,-3.643173e-02,1.841582e-02,-0.042228,-5.638744e-02,-3.282666e-03,-3.306697e-02,...,-1.161840e-03,1.595393e-03,2.138890e-03,2.547175e-03,-4.435927e-04,4.020542e-03,4.063547e-03,4.610419e-04,-3.387025e-03,-2.253409e-03
75%,2.028248,0.671133,8.434433e-01,0.634597,6.103616e-01,5.436087e-01,0.455046,3.840625e-01,3.983418e-01,3.010280e-01,...,4.917695e-02,4.810758e-02,5.010452e-02,4.972066e-02,4.851338e-02,4.975512e-02,4.730810e-02,4.612965e-02,4.245259e-02,4.358330e-02
max,9.400922,5.528548,4.758930e+00,5.382082,3.156900e+00,3.347648e+00,3.713261,2.702806e+00,3.812309e+00,2.124128e+00,...,4.027786e-01,4.702502e-01,3.853908e-01,2.958947e-01,3.627301e-01,2.956055e-01,2.759145e-01,2.924626e-01,3.119361e-01,3.055263e-01


In [47]:
#Try Random forest and other classifiers
rf_split = RandomForestClassifier(random_state=1337,criterion="entropy",max_depth=70, min_samples_split=15,n_estimators=1000) 
#rf_split.fit(x_full,y_full)
#Support Vector Classifier, C should actually be seperately tuned 
svm_split = sklearn.svm.SVC(random_state=1337,C=0.5) 
#Ridge Classifier, alpha should actually be seperately tuned
ridge_split = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)
#Ada Boost 
ada_split = sklearn.ensemble.AdaBoostClassifier(n_estimators=1000,random_state=1337)

In [48]:
scores_rf=cross_val_score(rf_split, x_full, y_full, n_jobs=-1,scoring=make_scorer(accuracy_score),  cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579))
scores_rf

array([0.63218391, 0.7816092 , 0.70114943, 0.56976744, 0.65116279,
       0.65116279, 0.60465116, 0.8255814 , 0.68604651, 0.61627907])

In [51]:
scores_rf.mean()

0.6719593691526331

In [49]:
scores_svm=cross_val_score(svm_split, x_full, y_full, cv= StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_svm

array([0.63218391, 0.73563218, 0.63218391, 0.56976744, 0.62790698,
       0.60465116, 0.60465116, 0.80232558, 0.58139535, 0.58139535])

In [50]:
scores_ridge=cross_val_score(ridge_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ridge

array([0.64367816, 0.7816092 , 0.70114943, 0.54651163, 0.61627907,
       0.69767442, 0.58139535, 0.75581395, 0.63953488, 0.60465116])

In [52]:
scores_ridge.mean()

0.6568297246725475

In [53]:
!pip install xgboost

In [54]:
scores_ada=cross_val_score(ada_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ada

array([0.62068966, 0.63218391, 0.6091954 , 0.48837209, 0.65116279,
       0.60465116, 0.65116279, 0.72093023, 0.56976744, 0.58139535])

In [55]:
#Now use XGBoost
import sklearn
import xgboost as xgb
xgb_split = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=10, min_child_weight=2, objective="binary:logistic", subsample=0.9, colsample_bytree=0.8, seed=1337)


In [56]:
scores_xgb=cross_val_score(xgb_split, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_xgb

array([0.64367816, 0.74712644, 0.63218391, 0.53488372, 0.6627907 ,
       0.6627907 , 0.61627907, 0.74418605, 0.59302326, 0.60465116])

In [57]:
print("Mean 10-fold CV Accuracy: ")
print("Random Forest: ",np.mean(scores_rf))
print("Support Vector Machine: ", np.mean(scores_svm))
print("Ridge Classifier: ",np.mean(scores_ridge))
print("Ada Boost: ", np.mean(scores_ada))
print("XGB Boost: ", np.mean(scores_xgb))

Mean 10-fold CV Accuracy: 
Random Forest:  0.6719593691526331
Support Vector Machine:  0.6372093023255815
Ridge Classifier:  0.6568297246725475
Ada Boost:  0.6129510825982358
XGB Boost:  0.6441593156909918


## Now look at Imputed Clinical Data and repeat the analysis

In [65]:
#####################
#Look at imputed clinical data
#import data

clinical_knn = pd.read_csv("trainSet_knn.txt")
clinical_imp = pd.read_csv("trainSet_imputed.txt")

clinical_knn.describe()
x_knn = clinical_knn.drop('Prognosis',1)
y_knn = clinical_knn[['Prognosis']]

x_imp = clinical_imp.drop('Prognosis',1)
y_imp = clinical_imp[['Prognosis']]
print(x_knn)
#y_knn = clinical_knn["Prognosis"]

     Unnamed: 0  Hospital        Age  Sex  Temp_C  Cough  \
0             0       2.0  35.913889  0.0   39.30    1.0   
1             1       2.0  57.266667  0.0   37.00    0.0   
2             2       2.0  79.263889  0.0   37.80    1.0   
3             3       2.0  82.000000  0.0   38.00    1.0   
4             4       2.0  60.791667  1.0   37.00    1.0   
..          ...       ...        ...  ...     ...    ...   
858         858       4.0  51.000000  0.0   37.51    0.0   
859         859       4.0  57.000000  0.0   37.40    1.0   
860         860       4.0  38.000000  0.0   37.14    0.0   
861         861       4.0  92.000000  1.0   38.00    0.0   
862         862       4.0  80.000000  0.0   38.40    0.0   

     DifficultyInBreathing    WBC     CRP  Fibrinogen    LDH  Ddimer  \
0                      0.0   5.76   43.40       651.0  387.0   157.0   
1                      0.0  11.48   64.00       700.0  338.0   601.0   
2                      0.0   6.21  115.30       698.0  356.0   

In [3]:
#Try Random forest and other classifiers
def ml_models ():
    rf_clinic = RandomForestClassifier(random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15,n_estimators=1000) 
    #rf_split.fit(x_full,y_full)
    #Support Vector Classifier, C should actually be seperately tuned 
    svm_clinic = sklearn.svm.SVC(random_state=1337,C=0.5) 
    #Ridge Classifier, alpha should actually be seperately tuned
    ridge_clinic = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)
    #Ada Boost 
    ada_clinic = sklearn.ensemble.AdaBoostClassifier(n_estimators=1000,random_state=1337)
    #XGB Boost
    #xgb_clinic = xgb.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=5, min_child_weight=2, objective="binary:logistic", subsample=0.9, colsample_bytree=0.8, seed=1337)
    return rf_clinic , svm_clinic, ridge_clinic, ada_clinic#, xgb_clinic

In [4]:
def ret_cv_scores(model,x_full,y_full):
    scores=cross_val_score(model, x_full, y_full, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
    return scores

In [61]:
rf_clinic_knn , svm_clinic_knn, ridge_clinic_knn, ada_clinic_knn = ml_models() #,xgb_clinic_knn

In [62]:
for i in [rf_clinic_knn , svm_clinic_knn, ridge_clinic_knn, ada_clinic_knn]:#, xgb_clinic_knn]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_knn, y_full = y_knn)
    print(np.mean(score_temp))
    

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.7579256883186314
SVC(C=0.5, random_state=1337)
0.634923817161187
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7416733493718258
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6952686447473939


In [63]:
rf_clinic_imp , svm_clinic_imp, ridge_clinic_imp, ada_clinic_imp = ml_models() #, xgb_clinic_imp

In [66]:
for i in [rf_clinic_imp , svm_clinic_imp, ridge_clinic_imp, ada_clinic_imp]:#, xgb_clinic_imp]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_imp, y_full = y_imp)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.745175086875167
SVC(C=0.5, random_state=1337)
0.5376770916867148
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7080459770114943
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6906041165463781


## Use Data from Autoencoder (not yet implemented)

In [181]:
#Try Autoencoded Data

clinical_y = pd.read_csv("Data Auto Encoder/trainSet.txt")
clinical_auto = pd.read_csv("Data Auto Encoder/simon_TRAIN_impute_classify.csv")

clinical_auto.describe()
x_auto = clinical_auto.drop(['PatientID','ImageFile','Hospital','Prognosis'],1)
y_auto = clinical_y[['Prognosis']]


In [182]:
x_auto

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure
0,35.913889,0.0,39.300000,1.0,0.0,5.76,43.40,651.000000,387.000000,157.000000,94.000000,71.863914,95.122742,7.651553,0.0,0.0
1,57.266667,0.0,37.000000,0.0,0.0,11.48,64.00,700.000000,338.000000,601.000000,94.000000,75.000000,96.900000,7.420000,0.0,0.0
2,79.263889,0.0,37.800000,1.0,0.0,6.21,115.30,698.000000,356.000000,448.000000,94.000000,63.000000,94.600000,7.390000,1.0,0.0
3,82.000000,0.0,38.000000,1.0,0.0,7.28,149.30,513.000000,482.000000,959.028015,97.000000,68.000000,96.300000,7.460000,0.0,0.0
4,60.791667,1.0,37.000000,1.0,0.0,6.37,20.70,577.426819,260.697968,210.000000,93.000000,74.015709,97.300000,7.639133,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,51.000000,0.0,38.222656,0.0,1.0,14.30,22.79,550.000000,368.000000,5027.000000,92.603195,70.067528,92.252815,7.590069,0.0,0.0
859,57.000000,0.0,38.613785,1.0,0.0,5.10,8.93,757.000000,451.000000,933.251465,95.931114,72.046349,95.225761,7.650484,0.0,0.0
860,38.000000,0.0,38.240093,0.0,1.0,7.30,0.23,611.945129,386.043854,1073.000000,92.770645,70.120804,92.430649,7.593248,0.0,0.0
861,92.000000,1.0,38.000000,0.0,1.0,13.50,3.77,533.000000,358.000000,2154.000000,93.311035,75.700000,93.616997,7.360000,0.0,0.0


In [183]:
rf_auto, svm_auto, ridge_auto, ada_auto = ml_models() #,xgb_clinic_knn

In [184]:
for i in [rf_auto, svm_auto, ridge_auto, ada_auto]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_auto, y_full = y_auto)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.7625233894680565
SVC(C=0.5, random_state=1337)
0.6594627105052125
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7312483293237102
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6848703555199145


In [ ]:
xgb_split = xgb.XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=10, min_child_weight=2, objective="binary:logistic", subsample=0.9, colsample_bytree=0.8, seed=1337)


In [151]:
#read in combined data
combined_imp = pd.read_pickle("df_full.p")
cnn_imp = pd.read_pickle("Xy_imgs.p")
#print(combined_imp.describe())
#print(combined_imp.shape)
x_combined = combined_imp.drop(['Prognosis','PatientID','ImageFile','Hospital'],1)

In [ ]:
y_combined = combined_imp["Prognosis"].apply(lambda x: 1 if x == "SEVERE" else 0)

In [153]:
cnn_fin = cnn_imp[0]

In [155]:
cnn_fin

array([[0.        , 0.        , 0.        , ..., 0.        , 0.05012614,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.03571007,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00273884, 0.02464284,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.13519804,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.24280955,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00892852,
        0.        ]], dtype=float32)

In [156]:
pca_cnn = decomposition.PCA(n_components = 100) #use 150 if you want more variance explained
pca_cnn.fit(cnn_fin)

PCA(n_components=100)

In [165]:
x_new = pca_all.transform(cnn_fin)
y_comb = cnn_imp[1]

In [172]:
x_comb = pd.concat([x_knn,pd.DataFrame(x_new)],axis=1)


In [162]:
rf_comb, svm_comb, ridge_comb, ada_comb = ml_models()

In [174]:
for i in [rf_comb, svm_comb, ridge_comb, ada_comb]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = x_comb, y_full = y_comb)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.6452419139267576
SVC(C=0.5, random_state=1337)
0.5063753007217322
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.6487703822507351
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.5897353648757017


## Use CNN only

In [175]:
rf_cnn, svm_cnn, ridge_cnn, ada_cnn = ml_models()

In [177]:
for i in [rf_cnn, svm_cnn, ridge_cnn, ada_cnn]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = pd.DataFrame(x_new), y_full = y_comb)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.642956428762363
SVC(C=0.5, random_state=1337)
0.625554664528201
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.6464314354450682
AdaBoostClassifier(n_estimators=1000, random_state=1337)
0.6048115477145148


## Final Prediction step

In [27]:
#load data kNN
train_knn = pd.read_csv("./data/train_imputed_knn20.csv")
train_raw = pd.read_csv ("./data/trainSet.txt")
y_knn= train_raw["Prognosis"].apply(lambda x: 1 if x == "SEVERE" else 0)

x_test = pd.read_csv ("./data/test_imputed_knn20.csv")

In [40]:
#Just some model CV:
rf_comb, svm_comb, ridge_comb, ada_comb = ml_models()

In [41]:
for i in [rf_comb, svm_comb, ridge_comb]:
    print(i)
    score_temp = ret_cv_scores(i,x_full = train_knn, y_full = y_knn)
    print(np.mean(score_temp))

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=15,
                       n_estimators=1000, random_state=1337)
0.746271050521251
SVC(C=0.5, random_state=1337)
0.6233761026463512
RidgeClassifier(alpha=2, normalize=True, random_state=1337)
0.7416332531408715


In [65]:
xgb_knn10 = xgb.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=10, min_child_weight=2, objective="binary:logistic", subsample=0.7, colsample_bytree=0.8, seed=1337)


In [66]:
scores_xgb10=cross_val_score(xgb_knn10, train_knn, y_knn, cv=StratifiedKFold(n_splits = 10, shuffle=True, random_state=1579),n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_xgb

array([0.64367816, 0.74712644, 0.63218391, 0.53488372, 0.6627907 ,
       0.6627907 , 0.61627907, 0.74418605, 0.59302326, 0.60465116])

In [67]:
scores_xgb.mean()

0.6441593156909918

In [23]:
rf_clinic = RandomForestClassifier(n_estimators=1000,random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15) 
rc_cnn = RandomForestClassifier(n_estimators=1000,random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15) 
#rf_split.fit(x_full,y_full)
#Support Vector Classifier, C should actually be seperately tuned 
svm_clinic = sklearn.svm.SVC(random_state=1337,C=0.5) 
#Ridge Classifier, alpha should actually be seperately tuned
ridge_clinic = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)

In [35]:
rf_clinic.fit(train_knn,y_knn)
rf_cnn.fit(x_)
svm_clinic.fit(train_knn,y_knn)
ridge_clinic.fit(train_knn,y_knn)

RidgeClassifier(alpha=2, normalize=True, random_state=1337)

In [68]:
pred_y = rf_clinic.predict(x_test)

In [69]:
pred_y

array([0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0], dtype=int64)

In [ ]:
pandas.to_csv(pred_y)

In [17]:
from sklearn.ensemble import VotingClassifier

In [33]:
eclf = VotingClassifier(estimators=[rf_clinic,svm_clinic,ridge_clinic],voting = 'hard')
eclf.fit(train_knn,y_knn)

TypeError: 'SVC' object is not iterable